In [ ]:
import os
import sys
import requests
from datetime import datetime 
from multiprocessing import Pool
import numpy as  np
import pandas as pd
from pathlib import Path
import pickle
from tqdm import tqdm
from time import time
from typing import Dict, List, Optional
sys.path.append('../')
sys.path.append('../anmodel')
import matplotlib.pyplot as plt
import anmodel
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.rcParams['font.family'] = 'Arial'

In [ ]:
S=pd.read_csv("A.csv")
anan1=anmodel.models.NAN20model()
num=37

A=[]
A.append(S["g_kvhh"][num])
A.append(S["g_unav"][num])
A.append(S["g_nak"][num])
A.append(S["g_leak"][num])
A.append(S["x_na"][num])
A.append(S["y_na"][num])
A.append(S["g_cav"][num])


BBB=['g_kvhh', 'g_unav', 'g_nak', 'g_leak','x_na', 'y_na', 'g_cav']
dic={key: val for key , val in zip(BBB,A)}
new_params=dic
new_params

In [ ]:
S=pd.read_csv("A.csv")
anan1=anmodel.models.NAN20dmodel()
num=37

A=[]
A.append(S["g_kvhh"][num])
A.append(S["g_unav"][num])
A.append(S["g_nak"][num])
A.append(S["g_leak"][num]*0.6095)
A.append(S["g_leak"][num]*0.3905)
A.append(S["x_na"][num])
A.append(S["y_na"][num])
A.append(S["g_cav"][num])


BBB=['g_kvhh', 'g_unav', 'g_nak', 'g_lek','g_lena', 'x_na', 'y_na', 'g_cav']
dic={key: val for key , val in zip(BBB,A)}
new_params=dic
anan1.set_params(new_params)
s,info=anan1.run_odeint()
plt.plot(s[400000:,0])

In [ ]:
new_params

In [ ]:
plt.plot(s[400000:,3])

In [ ]:
fig=plt.figure()
ax = fig.add_subplot(111)
plt.plot(s[450000:,0])
Z=np.linspace(-80,20, 5)
ax.set_yticks(Z)
plt.savefig("Fig4B.pdf")

In [ ]:
fig=plt.figure()
ax = fig.add_subplot(111)
plt.plot(s[450000:,3])
Z=np.linspace(7.4,8.2, 5)
ax.set_yticks(Z)
plt.savefig("Fig4C.pdf")

In [ ]:
t=[]
for i in range(1*300*10):
    t.append(0.001*0.1*i)
C=[]
D=[]
for i in range(len(t)):
    C.append(s[389000:,0][i])
    D.append(s[389000:,3][i])
fig=plt.figure()
ax = fig.add_subplot(111)
plt.plot(t,C, linewidth=1)
Z=np.linspace(-80,20,5)
ax.set_yticks(Z)
plt.savefig("Fig4F.pdf")

In [ ]:
t1=[]
for i in range(int(0.1*1000*10)):
    t1.append(0.001*0.1*i)
C1=[]
D1=[]
for i in range(len(t1)):
    C1.append(s[455900:,0][i+1400])
    D1.append(s[389000:,3][i+1400])
fig=plt.figure()
ax = fig.add_subplot(111)
plt.plot(t1,C1, linewidth=1)
Z=np.linspace(-80,20,5)
ax.set_yticks(Z)
Z=np.linspace(0,0.1,5)
ax.set_xticks(Z)
plt.savefig("Fig4G.pdf")

In [ ]:
#Current
#IUNaV. INaV. INaP, IUNaP, IKNa, IKvhh, ILeak, ICaV
VK=-100.0
VLeak=-60.95
VNa=55.0
VCa=120.0

#INaV
def am(v):
    if v == -33.:
        return 1.
    else:
        return 0.1 * (v+33.0) / (1.0-np.exp(-(v+33.0)/10.0))
def bm(v):
    return 4.0 * np.exp(-(v+53.7)/12.0)

def mna3(v):
    xxx1xxx=am(v)/(am(v)+bm(v))
    return xxx1xxx**3

def INaV(g, v,h):
    return g*mna3(v)*h*(v-VNa)


#ILeak

def ILeak(g,v):
    return g*(v-VLeak)


def ILeakK(g,v):
    return g*0.6095*(v-VK)
def ILeakNa(g,v):
    return g*0.3905*(v)

#IKvhh

def IKvhh(g, v,n):
    return g*(n**4)*(v-VK)
                     
def mca(v):
    return (1.0 / (1.0 + np.exp(-(v+20.0)/9.0)))**2

def ICaV(g, v):
    return g*mca(v)*(v-VCa)


#IKNa
def mna(na):
    return 1.0/(1.0+(32.0/na)**(3.0))


def IKNa(g,v,na):
    return g*mna(na)*(v-VK)

def apump(na):
    return ((1+3.5/4.0)**-2)*((1+10.0/na)**-3)

def INaK(g,na):
    return g*apump(na)

#IUNaV

def amuv(v,x):
    if v == -33.:
        return 1.
    else:
        return 0.1 * (v+33.0+x) / (1.0-np.exp(-(v+33.0+x)/10.0))
def bmuv(v,x):
    return 4.0 * np.exp(-(v+53.7+x)/12.0)

def mnauv3(v,x):
    xxx1xxx=amuv(v,x)/(amuv(v,x)+bmuv(v,x))
    return xxx1xxx**3

def IUNaV(g,v,h,x):
    return g*mnauv3(v,x)*h*(v-VNa)



gK= 90.22913406
gUNaV=18.22838513
gNaK=98.68629964
gLeak=0.074996331
gCaV=0.039755106
x=29.9540276
y=15.91732198


In [ ]:
fig=plt.figure()
ax = fig.add_subplot(111)
plt.plot(t,Na, linewidth=1)
Z=np.linspace(8.14,8.26,5)
ax.set_yticks(Z)
Z=np.linspace(0,0.1,5)
ax.set_xticks(Z)
plt.savefig("Fig4J.pdf")

In [ ]:
UNAV=[]
LENA=[]
CAV=[]

LEK=[]
KVHH=[]
NAK=[]

t=[]
for i in range(1000):
    t.append(0.001*0.1*i)
unavN=[]
lenaN=[]
cavN=[]
lekN=[]
kvhhN=[]
nakN=[]

Na=[]

tNAC=[]
IN=[]
for i in range(len(t)):
    UNAV.append(IUNaV(gUNaV, s[390400:,0][i], s[390400:,1][i], x))
    LENA.append(ILeakNa(gLeak, s[390400:,0][i]))
    CAV.append(ICaV(gCaV, s[390400:,0][i]))
    
    LEK.append(ILeakK(gLeak, s[390400:,0][i]))
    KVHH.append(IKvhh(gK, s[390400:,0][i], s[390400:,2][i]))
    NAK.append(INaK(gNaK, s[390400:,3][i]))
    IN.append((-UNAV[i]-gLeak*(100-60.95)/100*0.44*(s[390400:,0][i]-55.0))/5000)

    tNAC.append(1000*3*INaK(gNaK, s[390400:,3][i])/5000)
    Na.append(s[390400:,3][i])
    
    unavN.append(abs(UNAV[i])/(abs(UNAV[i])+abs(LENA[i])+abs(CAV[i])))
    lenaN.append(abs(LENA[i])/(abs(UNAV[i])+abs(LENA[i])+abs(CAV[i])))
    cavN.append(abs(CAV[i])/(abs(UNAV[i])+abs(LENA[i])+abs(CAV[i])))
    lekN.append(abs(LEK[i])/(abs(LEK[i])+abs(KVHH[i])+abs(NAK[i])))
    kvhhN.append(abs(KVHH[i])/(abs(LEK[i])+abs(KVHH[i])+abs(NAK[i])))
    nakN.append(abs(NAK[i])/(abs(LEK[i])+abs(KVHH[i])+abs(NAK[i])))
    
    
    


In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(t,IN, linewidth=1, label="IN", color="red")
ax2.plot(t,tNAC, linewidth=1, label="OUT", color="blue")
Z=np.linspace(0,0.2,5)
ax1.set_yticks(Z)
Z=np.linspace(0,0.1,5)
ax1.set_xticks(Z)
Z=np.linspace(1.52,1.55,4)
ax2.set_yticks(Z)
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines1 + lines2, labels1 + labels2, loc='upper right')
plt.savefig("Fig4K.pdf")


In [ ]:
t=[]
for i in range(1000):
    t.append(0.001*0.1*i)
fig=plt.figure()
ax = fig.add_subplot(111)

plt.plot(t,cavN, linewidth=1, color="orange", label="CaV")

Z=np.linspace(0,1,5)
ax.set_yticks(Z)
Z=np.linspace(0,0.1,5)
ax.set_xticks(Z)
plt.legend()

In [ ]:
fig=plt.figure()
ax = fig.add_subplot(111)
plt.plot(t,unavN, linewidth=1, color="red", label="UNaV")
plt.plot(t,cavN, linewidth=1, color="orange", label="CaV")
plt.plot(t,lenaN, linewidth=1, color="yellow", label="LeakNa")
Z=np.linspace(0,1,5)
ax.set_yticks(Z)
Z=np.linspace(0,0.1,5)
ax.set_xticks(Z)
plt.legend()
plt.savefig("Fig4H.pdf")

In [ ]:
fig=plt.figure()
ax = fig.add_subplot(111)
plt.plot(t,lekN, linewidth=1, color="purple", label="LeakK")
plt.plot(t,kvhhN, linewidth=1, color="blue", label="Kvhh")
plt.plot(t,nakN, linewidth=1, color="green", label="NaK")
Z=np.linspace(0,1,5)
ax.set_yticks(Z)
Z=np.linspace(0,0.1,5)
ax.set_xticks(Z)
plt.legend()
plt.savefig("Fig4I.pdf")